In [1]:
#%pip install beautifulsoup4
#%pip install pip-system-certs

In [15]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import stardog
import os
import io
from langchain_community.graphs import Neo4jGraph
from PyPDF2 import PdfReader
from langdetect import detect
import spacy
import re
from unidecode import unidecode

Conectando ao Neo4j 

In [16]:
from neo4j import GraphDatabase

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")


graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

- Buscar os links para a tese no Neo4j
- Buscar o link do PDF no repositório da universidade
- Baixar o PDF
- Extrair o texto do PDF 

Buscar links para a tese no Neo4j

In [ ]:
# Query to get theses and their repositories from the Neo4j database
# This query retrieves theses that are published by the University of São Paulo (USP) and have not been processed yet.
query1 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:USP'})
WHERE NOT EXISTS {(t{processed:true})}
RETURN distinct(t["uri"]) as Thesis, t["repository"] as repository 
"""

# Alternative query to get theses that do not have excerpts and have been processed.
query2 = """ 
match (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:USP'})
where (not (t:Thesis)-[:BFO_0000050]-(:Exerpt)) and  exists {(t{processed:true})}
return distinct(t["uri"]) as Thesis, t["repository"] as repository
"""

# Query to retrive theses with few pages and their repositories.
query3 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:USP'})
Match (t:Thesis)-[:BFO_0000050]-(e:Exerpt)
RETURN count(e) as pages, t["uri"] as Thesis, t["repository"] as repository
 order by (pages) limit 40
"""

graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
results = graph.execute_query(query3).records
graph.close()

#results = graph.query(query2)
# Convert the results to a DataFrame
thesis_repository = pd.DataFrame(results, columns=["pages", "Thesis", "repository"])
#thesis_repository = pd.DataFrame(results, columns=["Thesis", "repository"])
thesis_repository = thesis_repository.reset_index(drop=True) 
thesis_repository

,pages,Thesis,repository
0,1,tag:stardog:api:comparison_of_fobusp_standard_...,https://www.teses.usp.br/teses/disponiveis/25/...
1,1,tag:stardog:api:debates_e_projetos_sobre_escra...,https://www.teses.usp.br/teses/disponiveis/8/8...
2,1,tag:stardog:api:o_espaco_humanizado_a_paisagem...,http://www.teses.usp.br/teses/disponiveis/8/81...
3,1,tag:stardog:api:historias_de_ebano_professoras...,http://www.teses.usp.br/teses/disponiveis/48/4...
4,1,tag:stardog:api:segmentacao_no_mercado_de_trab...,http://www.teses.usp.br/teses/disponiveis/11/1...
5,2,tag:stardog:api:mulheres_negras_em_profissoes_...,https://www.teses.usp.br/teses/disponiveis/48/...
6,2,tag:stardog:api:avaliacao_sanitaria_da_presenc...,http://www.teses.usp.br/teses/disponiveis/91/9...
7,2,tag:stardog:api:petrologia_e_mineralogia_de_ro...,http://www.teses.usp.br/teses/disponiveis/44/4...
8,2,tag:stardog:api:capacidades_de_inovacao_em_neg...,https://www.teses.usp.br/teses/disponiveis/96/...
9,2,tag:stardog:api:a_conservacao_da_paisagem_como...,http://www.teses.usp.br/teses/disponiveis/90/9...


Criando as utils functions

In [55]:

# Funcao para coletar link do pdf

def get_pdf_link(url):
    
    #Fazer requisição e parsear o arquivo html
    f = requests.get(url, verify=True).text#, verify=False) 
    soup = bs(f, "html.parser")
    
    #Coletando link para as teses
    for doc in soup.find_all('a', href=True):
        if doc['href'].endswith('.pdf'):
            path = doc['href']
            prefix_uni = 'https://www.teses.usp.br'
            link = prefix_uni + path

            return link
    
    return None

#fazer download do arquivo
def download_file(pdf_link, thesis_uri):
    # NOTE the stream=True parameter below
    r = requests.get(pdf_link, verify=True, stream=True)
    # raise_for_status() is not needed with stream=True
    # r.raise_for_status()
    with open('tese 2.pdf', 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return 

#Carregando os modelos SpaCy para inglês e português 
nlp_en = spacy.load("en_core_web_lg")
nlp_pt = spacy.load("pt_core_news_lg")

# Funcão que recebe uma string e a limpar para ficar no formato aceitável para uma URI
def process_uri(x):
    return (re.sub('[^a-zA-Z0-9_ ]', '',
            unidecode(x.strip())
                        .replace(" ", "_")
                        .replace("[","")
                        .replace("]","")
                        .replace("?","")
                        .replace("'","")
                        .lower()))


# Função para extrair texto de uma tese em PDF e identificar entidades
def extratc_text(tese_path):
    
    # Processando o arquivo em PDF
    try:
        reader = PdfReader(tese_path)
    except:
        print(f"Erro ao ler o arquivo {tese_path}. ")
        excerpt = {}
        return excerpt
    number_of_pages = len(reader.pages)
    excerpt = {}
    

    # Iterando por cada página
    for page_number in range(number_of_pages):
        
        
        # Extraindo o texto da página
        try:
            if page_number == 0:
                try:
                    page_text = (reader.pages[page_number].extract_text() + '\n ' 
                                + reader.pages[page_number+1].extract_text()[:400])
                except:
                    page_text = reader.pages[page_number].extract_text()
            else:
                
                if page_number == number_of_pages -1:
                    try:
                        page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                                    + reader.pages[page_number].extract_text())
                    except:
                        page_text = reader.pages[page_number].extract_text()
                
                else:
                    try:
                        page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                                    + reader.pages[page_number].extract_text() + '\n ' 
                                    + reader.pages[page_number+1].extract_text()[:400])  
                    except:
                        page_text = reader.pages[page_number].extract_text()
        except:
            page_text = ''
        
        # Limpando page text
        page_text = page_text.replace("'", "").replace('"', '')
        # Verifica se o último caractere é uma barra invertida
        if page_text != '':
            if page_text[-1] == '\\': 
                page_text = page_text[:-1]
        page_text = re.sub(r"[\ud800-\udfff]", " ", page_text)
        page_text = page_text.encode('utf-8')
        
        # Detectando o idioma do texto
        try:
            lang = detect(page_text)
            
        except:
            lang = 'Não_detectado'
        
        # Extraindo as entidades
        page_extracted = False
        if lang == 'pt':
            doc = nlp_pt(page_text)
            page_extracted = True
        if lang == 'en':
            doc = nlp_en(page_text)
            page_extracted = True
        
        if page_extracted:

            persons = []
            gpes = []
            #orgs = []
            for ent in doc.ents:
                if ent.label_ == "PER": #"PERSON":
                    persons.append(process_uri(ent.text))
                if ent.label_ == "LOC": #"GPE":
                    gpes.append(process_uri(ent.text))
                #if ent.label_ == "ORG":
                #    orgs.append(process_uri(ent.text))
                    
            excerpt[page_number] = {'text': page_text, 
                                    'lang': lang, 
                                    'persons': list(set(persons)), 
                                    'gpes': list(set(gpes)),
                                #'orgs': list(set(orgs))
                                }

    return excerpt

Iterando por todas as URI para extrair o texto e gravar no grafo NEO4j

In [56]:
# Gravar hora de início do processamento
start_time = pd.Timestamp.now()
parcial_time = pd.Timestamp.now()

for i in range(0,len(thesis_repository)):

    # Coletando o link do PDF
    url = thesis_repository['repository'][i]
    thesis_uri = thesis_repository['Thesis'][i][16:]
    pdf_link = get_pdf_link(url)
    if pdf_link is not None:
        print(f"{i} - Downloading and extracting: {thesis_uri} from {pdf_link}")
        download_file(pdf_link, thesis_uri)
        
        # Extraindo o texto e entidades
        tese_path = 'tese 2.pdf'
        excerpt = extratc_text(tese_path)

        excerpts = []
        persons = []
        gpes = []

        # Salvando as triplas em CSV
        for n in excerpt.keys():

            excerpt_uri = thesis_repository['Thesis'][i] + '_p_' + str(n)
            #excerpt_text = str(excerpt[n]['text']).replace("'", "").replace('"', '')
            # Verifica se o último caractere é uma barra invertida
            #if excerpt_text[-1] == '\\': 
            #    excerpt_text = excerpt_text[:-1]
            

            excerpt_page = n
            excerpt_lang = excerpt[n]['lang']
            
            excerpts.append([thesis_repository['Thesis'][i], excerpt_uri, excerpt_text, str(excerpt_page), excerpt_lang])
                
            #Criando as relações com as entidades
            for person in excerpt[n]['persons']:
                persons.append([excerpt_uri, 'tag:stardog:api:' + person])
            
            for gpe in excerpt[n]['gpes']:
                gpes.append([excerpt_uri, 'tag:stardog:api:' + gpe])
        
        # Salvando os dados em CSV no diretório de importação do Neo4j
        pd.DataFrame(excerpts).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\excerpts2.csv', index=False, header=False)
        pd.DataFrame(persons).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\persons2.csv', index=False, header=False)
        pd.DataFrame(gpes).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\gpes2.csv', index=False, header=False)
        
        #Deletando o arquivo PDF após o processamento
        os.remove('tese 2.pdf')

        print(f"Loading triplas: {thesis_uri}")

        # Importanto os dados para o Neo4j
        #Excerpts
        query = """
        LOAD CSV FROM 'file:///excerpts2.csv'
        AS row
        MATCH  (t:Thesis{uri: row[0]})
        MERGE  (e:Exerpt{
                uri: row[1],
                text: row[2],
                page: row[3],
                lang: row[4]
                })-[r:BFO_0000050]-(t);
        """
        graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
        results = graph.execute_query(query)
        graph.close()

        #Persons
        query = """
        LOAD CSV FROM 'file:///persons2.csv'
        AS row
        MATCH  (e:Exerpt{uri: row[0]})
        MERGE  (p:Person{uri: row[1]})
        MERGE  (e)-[r:mentions]-(p);
        """
        graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
        results = graph.execute_query(query)
        graph.close()

        #GPEs
        query = """
        LOAD CSV FROM 'file:///gpes2.csv'
        AS row
        MATCH  (e:Exerpt{uri: row[0]})
        MERGE  (p:Place{uri: row[1]})
        MERGE  (e)-[r:mentions]-(p);
        """
        graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
        results = graph.execute_query(query)
        graph.close()

        # Removendo os arquivos CSV após a importação
        os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\excerpts2.csv')
        os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\persons2.csv')
        os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\gpes2.csv')

        print(f"Total time: {pd.Timestamp.now() - start_time}")
        print(f"Partial time: {pd.Timestamp.now() - parcial_time}")
        # Atualizando a hora parcial
        parcial_time = pd.Timestamp.now()
    else:
        print(f"No PDF link found for {thesis_uri}")

    # gravando no grafo que a tese foi processada
    query = """
    MATCH  (t:Thesis{uri: \'""" + thesis_repository['Thesis'][i] +  """\'})
    SET t.processed = true
    """
    graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    results = graph.execute_query(query)
    graph.close()

0 - Downloading and extracting: comparison_of_fobusp_standard_cephalometric_analysis_in_caucasian_black_japanese_afrocaucasian_and_japanese_caucasian_individuals from https://www.teses.usp.br/teses/disponiveis/25/25144/tde-02092024-163215/publico/LauraDiasSoviero_resumida.pdf
Loading triplas: comparison_of_fobusp_standard_cephalometric_analysis_in_caucasian_black_japanese_afrocaucasian_and_japanese_caucasian_individuals
Total time: 0 days 00:00:02.465348
Partial time: 0 days 00:00:02.465348
1 - Downloading and extracting: debates_e_projetos_sobre_escravidao_na_decada_de_1860 from https://www.teses.usp.br/teses/disponiveis/8/8138/tde-18112020-201659/publico/2020_BrunaOliveiraDaSilva_Errata.pdf
Loading triplas: debates_e_projetos_sobre_escravidao_na_decada_de_1860
Total time: 0 days 00:00:04.351523
Partial time: 0 days 00:00:01.886175
2 - Downloading and extracting: o_espaco_humanizado_a_paisagem_humanizada_e_algumas_reflexoes_sobre_a_paisagem_em_sao_paulo_na_primeira_metade_do_seculo_xi

Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x29b for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x29b for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x29b for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x29b for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x29b for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x29b for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitions in dictionary at byte 0x29b for key /Subtype
Multiple definitions in dictionary at byte 0x7a for key /Subtype
Multiple definitio

Loading triplas: respuesta_clinica_frente_a_los_antibioticos_de_los_ninos_indigenas_y_no_indigenas_menores_de_cinco_anos_con_neumonia_adquirida_en_la_comunidad_en_otavaloecuador
Total time: 0 days 00:03:31.734604
Partial time: 0 days 00:00:10.724082
21 - Downloading and extracting: movimiento_hip_hop_estudo_comparativo_entre_cuba_y_brasil from https://www.teses.usp.br/teses/disponiveis/84/84131/tde-07122018-170333/publico/2017_DaniaChavianoSuarez_VOrig.pdf
Loading triplas: movimiento_hip_hop_estudo_comparativo_entre_cuba_y_brasil
Total time: 0 days 00:03:52.284777
Partial time: 0 days 00:00:20.550173
22 - Downloading and extracting: musica_educacion_y_descolonizacion_el_modelo_educativo_sociocomunitario_productivo_del_estado_plurinacional_de_bolivia_en_el_contexto_de_las_culturas_populares_de_america_latina_y_el_caribe from https://www.teses.usp.br/teses/disponiveis/84/84131/tde-04082021-192439/publico/2021_FelixCenevivaEid_VCorr.pdf
Loading triplas: musica_educacion_y_descolonizacion_